In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    cohen_kappa_score,
    classification_report,
)
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Путь к объединенным CSV-файлам
combined_path = "./combined_csvs/"
combined_files = [f for f in os.listdir(combined_path) if f.endswith(".csv")]

# Сортировка файлов по числу пакетов (например, _250_)
combined_files.sort(key=lambda f: int(re.search(r"(\d+)_packets", f).group(1)))

# Словарь моделей с параметром verbose
models = {
    "Random Forest": RandomForestClassifier(random_state=42, verbose=1, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42, verbose=1),
    "SVM": SVC(probability=True, verbose=1, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "Logistic Regression": LogisticRegression(
        solver="saga",
        multi_class="multinomial",
        class_weight="balanced",
        max_iter=1000,
        verbose=1,
        n_jobs=-1,
        random_state=42,
    ),
}

# Создание папки для промежуточных файлов
intermediate_path = "./multiclass_models_results/"
os.makedirs(intermediate_path, exist_ok=True)

# Итоговый список для всех результатов
all_results = []

# Проход по каждому CSV-файлу
for combined_file in tqdm(combined_files, desc="Обработка файлов"):
    df = pd.read_csv(os.path.join(combined_path, combined_file))
    X = df.drop(columns=["Label"])
    y = df["Label"]

    # Извлечение размера пакетов из имени файла
    packet_size = int(re.search(r"(\d+)_packets", combined_file).group(1))

    # Разделение данных и нормализация в отдельном блоке try
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    except ValueError as e:
        print(f"  ! Ошибка при подготовке данных для файла '{combined_file}': {e}")
        continue

    # Проход по каждой модели
    for model_name, model in models.items():
        results = {
            "Model": model_name,
            "File": combined_file,
            "Packet Size": packet_size,
        }

        print(f"  - Обучение модели {model_name}")
        try:
            # Обучение модели с выводом прогресса
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Расчёт метрик
            results["Accuracy"] = accuracy_score(y_test, y_pred)
            results["Macro F1 Score"] = f1_score(y_test, y_pred, average="macro")
            results["Micro F1 Score"] = f1_score(y_test, y_pred, average="micro")
            results["Weighted F1 Score"] = f1_score(y_test, y_pred, average="weighted")
            results["Macro Precision"] = precision_score(
                y_test, y_pred, average="macro"
            )
            results["Micro Precision"] = precision_score(
                y_test, y_pred, average="micro"
            )
            results["Cohen's Kappa"] = cohen_kappa_score(y_test, y_pred)

            # Отчёт по каждому классу
            class_report = classification_report(y_test, y_pred, output_dict=True)
            for label, metrics in class_report.items():
                if isinstance(metrics, dict):
                    results[f"Precision_{label}"] = metrics["precision"]
                    results[f"Recall_{label}"] = metrics["recall"]
                    results[f"F1_{label}"] = metrics["f1-score"]

        except ValueError as e:
            # Пропуск метрик при ошибке
            results.update(
                {
                    "Accuracy": "0.0",
                    "Macro F1 Score": "0.0",
                    "Micro F1 Score": "0.0",
                    "Weighted F1 Score": "0.0",
                    "Macro Precision": "0.0",
                    "Micro Precision": "0.0",
                    "Cohen's Kappa": "0.0",
                }
            )
            print(
                f"  ! Ошибка обучения модели '{model_name}' для файла '{combined_file}': {e}"
            )

        # Добавление результата
        all_results.append(results)

        # Промежуточный CSV для модели
        model_results_df = pd.DataFrame([results])
        model_results_file = os.path.join(
            intermediate_path, f"{model_name}_multiclass_results.csv"
        )

        # Если файл существует, добавляем данные
        if os.path.exists(model_results_file):
            model_results_df.to_csv(
                model_results_file, mode="a", index=False, header=False, float_format="%.3f"
            )
        else:
            model_results_df.to_csv(model_results_file, index=False, float_format="%.3f")

# Итоговый файл с результатами всех моделей
final_results_df = pd.DataFrame(all_results)
final_results_df.to_csv("final_multiclass_models_results.csv", index=False, float_format="%.3f")
print("Итоговый файл сохранён в 'final_multiclass_models_results.csv'")

Обработка файлов:   0%|          | 0/9 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.


  - Обучение модели Random Forest


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    0.1s finished


  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.4967            2.01m
         2           1.7190            1.96m
         3           1.6196            1.92m
         4           1.5303            1.89m
         5           1.4644            1.87m
         6           1.4074            1.85m
         7           1.3610            1.82m
         8           1.3183            1.80m
         9           1.2862            1.78m
        10           1.2568            1.76m
        20         319.8759            1.56m
        30         319.8218            1.36m
        40         319.7953            1.17m
        50 48294246952245688.0000           58.48s
        60 48294246952245688.0000           46.79s
        70 48294246952245688.0000           35.10s
        80 48294246952245688.0000           23.39s
        90 48294246952245688.0000           11.69s
       100 48294246952245688.0000            0.00s
  - Обучение модели SVM
[

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.


...

  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.2140            0.32s
         2           0.9766            0.34s
         3           0.8047            0.34s
         4           0.6708            0.35s
         5           0.5645            0.34s
         6           0.4724            0.34s
         7           0.3944            0.33s
         8           0.3306            0.33s
         9           0.2780            0.33s
        10           0.2350            0.32s
        20           0.0426            0.28s
        30           0.0080            0.25s
        40           0.0015            0.21s
        50           0.0003            0.18s
        60           0.0001            0.14s
        70           0.0000            0.11s
        80           0.0000            0.07s
        90           0.0000            0.04s
       100           0.0000            0.00s
  - Обучение модели SVM
[LibSVM]*
optimization finished, #ite

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.


h 141, change: 0.00031430
Epoch 142, change: 0.00030816
Epoch 143, change: 0.00030232
Epoch 144, change: 0.00029726
Epoch 145, change: 0.00029088
Epoch 146, change: 0.00028181
Epoch 147, change: 0.00027751
Epoch 148, change: 0.00027111
Epoch 149, change: 0.00026575
Epoch 150, change: 0.00025979
Epoch 151, change: 0.00025381
Epoch 152, change: 0.00024974
Epoch 153, change: 0.00024419
Epoch 154, change: 0.00024004
Epoch 155, change: 0.00023391
Epoch 156, change: 0.00022857
Epoch 157, change: 0.00022464
Epoch 158, change: 0.00021993
Epoch 159, change: 0.00021648
Epoch 160, change: 0.00021048
Epoch 161, change: 0.00020753
Epoch 162, change: 0.00020145
Epoch 163, change: 0.00019743
Epoch 164, change: 0.00019403
Epoch 165, change: 0.00019000
Epoch 166, change: 0.00018611
Epoch 167, change: 0.00018293
Epoch 168, change: 0.00017792
Epoch 169, change: 0.00017539
Epoch 170, change: 0.00017395
Epoch 171, change: 0.00016722
Epoch 172, change: 0.00016455
Epoch 173, change: 0.00016036
Epoch 174, cha

Обработка файлов: 100%|██████████| 9/9 [15:28<00:00, 103.21s/it]

Итоговый файл сохранён в 'final_multiclass_models_results.csv'


In [3]:
# Создание папки для модифицированных промежуточных файлов
intermediate_path_modified = "./multiclass_models_results_modified/"
os.makedirs(intermediate_path_modified, exist_ok=True)

# Путь к промежуточным файлам
intermediate_path = "./multiclass_models_results/"
model_results_files = [f for f in os.listdir(intermediate_path) if f.endswith("_multiclass_results.csv")]

# Создаем итоговый DataFrame для всех моделей
final_results = []

# Процесс обработки каждого промежуточного файла модели
for model_results_file in model_results_files:
    # Чтение промежуточного CSV файла
    df = pd.read_csv(os.path.join(intermediate_path, model_results_file))
    # display(df)

    # Получаем список метрик, исключая "Packet Size" и "Model"
    metrics_columns = [col for col in df.columns if col not in ['Packet Size', 'Model', 'File']]

    # Определение лучшей выборки для каждой метрики
    best = {}
    counts = {}

    # Добавление столбца Counts для каждой выборки
    counts_column = []
    for index, row in df.iterrows():
        count = sum(1 for col in metrics_columns if row[col] == df[col].max())  # Считаем, сколько раз максимальное значение встречается
        counts_column.append(count)

    df["Counts"] = counts_column

    # Для каждой метрики находим наилучшие выборки
    for col in metrics_columns:  # Перебираем только метрики
        max_value = df[col].max()  # Находим максимальное значение для метрики
        best[col] = df[df[col] == max_value]["Packet Size"].tolist()  # Сохраняем размеры выборок с максимальным значением

        # Подсчитаем, сколько раз каждая выборка была максимальной для этой метрики
        counts[col] = df[col].value_counts().get(max_value, 0)  # Подсчитаем сколько раз максимальное значение встречается

    # Строка Best
    best_row = {"Packet Size": "Best"}
    for col in metrics_columns:
        best_row[col] = best[col]  # Присваиваем лучшие выборки для каждой метрики

    # Добавление строки Best в DataFrame
    df_best = pd.DataFrame([best_row])

    # Добавление строки Best в DataFrame
    df = pd.concat([df, df_best], ignore_index=True)
    
    model_results_file_modified = os.path.join(
            intermediate_path_modified, f"{os.path.splitext(model_results_file)[0]}_modified.csv"
        )

    # Запись результатов в новый CSV файл
    df.to_csv(model_results_file_modified, index=False)
